# Using AXI GPIO with PYNQ

## Goal

The aim of this notebook is to show how to use AXI GPIO from PYNQ. 

Multiple AXI GPIO controllers can be implemented in the programmable logic and used to control internal or external GPIO signals.

## Hardware design

This example uses a bitstream that connects three AXI GPIO controllers to the LEDs, buttons, and switches and can be used with the PYNQ-Z1 or PYNQ-Z2 board. (Each AXI GPIO controller has 2 channels, so multiple peripherals could be controlled from one AXI GPIO IP, but for simplicity and demonstration purposes, separate AXI GPIO controllers are used. 

![AXI GPIO Design](./images/axi_gpio_design.png "AXI GPIO Design")

### 1. Download the tutorial overlay

The `axi_gpio.bit` and `axi_gpio.tcl` files can be found in the bitstreams directory local to this folder. 
The bitstream can be downloaded by passing the relative path to the Overlay class. 

* Check the bitstream and .tcl exists in the bitstream directory

In [ ]:
!dir ./bitstream

* Download the bitstream

In [ ]:
from pynq import Overlay
axi_gpio_design = Overlay("./bitstream/axi_gpio.bit")

Check the IP Dictionary for this design. The IP dictionary lists AXI IP in the design, and for this example will list the AXI GPIO controllers for the buttons, leds, and switches. The Physical address, the address range and IP type will be listed. If any interrupts, or GPIO were connected to the PS, they would also be reported. 

In [ ]:
axi_gpio_design.ip_dict

## AxiGPIO class

The PYNQ AxiGPIO class will be used to access the AXI GPIO controllers. 

### 1. Controlling the switches and push-buttons

The instances can be found and referenced from the IP dictionary. 

In [ ]:
from pynq.lib import AxiGPIO

buttons_instance = axi_gpio_design.ip_dict['buttons']
buttons = AxiGPIO(buttons_instance).channel1

In [ ]:
buttons.read()

The buttons controller is connected to all four user push-buttons on the board (BTN0 to BTN3). Try pressing any combination of the buttons and rerunning the cell above.

The AXI GPIO controller for the switches can be used in a similar way:

In [ ]:
switches_instance = axi_gpio_design.ip_dict['switches']
switches = AxiGPIO(switches_instance).channel1

In [ ]:
print(f"Switches: {switches.read()}")

### 2. Controlling the LEDs

The LEDs can be used in a similar way. 

In [ ]:
from pynq.lib import AxiGPIO
led_instance = axi_gpio_design.ip_dict['leds']
led = AxiGPIO(led_instance).channel1

The outputs can be addressed using a slice. 

In [ ]:
led[0:4].write(0x1)

In [ ]:
from time import sleep

led[0:4].write(0x3)
sleep(1)
led[0:4].write(0x7)
sleep(1)
led[0:4].write(0xf)

* Reset the LEDs

In [ ]:
led[0:4].off()

### 3 Putting it together

Run a loop to set the LEDs to the value of the pushbuttons. 

Before executing the next cell, make sure Switch 0 (SW0) is "on". While the loop is running, press a push-button and notice the corresponding LED turns on. To exist the loop, change Switch 0 to off. 

In [ ]:
while(switches.read() is 1):
    led[0:4].write(buttons.read())